# Agents Azure AI avec prise en charge du protocole de contexte de modèle (MCP) - Python

Ce notebook montre comment utiliser les agents Azure AI avec les outils du protocole de contexte de modèle (MCP) en Python. Il explique comment créer un agent intelligent capable de tirer parti de serveurs MCP externes (comme Microsoft Learn) pour des fonctionnalités avancées en utilisant une authentification sans clé.


## Installer les packages Python requis

Tout d'abord, nous devons installer les packages Python nécessaires :
- **azure-ai-projects** : SDK principal pour les projets Azure AI
- **azure-ai-agents** : SDK Azure AI Agents pour créer et gérer des agents
- **azure-identity** : Fournit une authentification sans clé en utilisant DefaultAzureCredential
- **mcp** : Implémentation du protocole Model Context pour Python


## Avantages de l'authentification sans clé

Ce notebook illustre les avantages de l'**authentification sans clé** :
- ✅ **Pas de clés API à gérer** - Utilise l'authentification basée sur l'identité Azure
- ✅ **Sécurité renforcée** - Aucun secret stocké dans le code ou les fichiers de configuration
- ✅ **Rotation automatique des identifiants** - Azure gère le cycle de vie des identifiants
- ✅ **Contrôle d'accès basé sur les rôles** - Utilise Azure RBAC pour des permissions granulaires
- ✅ **Support multi-environnements** - Fonctionne parfaitement entre développement et production

Le `DefaultAzureCredential` sélectionne automatiquement la meilleure source d'identifiants disponible :
1. **Identité managée** (lors de l'exécution dans Azure)
2. Identifiants **Azure CLI** (pendant le développement local)
3. Identifiants **Visual Studio**
4. **Variables d'environnement** (si configurées)
5. Authentification via **navigateur interactif** (en dernier recours)


## Configuration de l'authentification sans clé

**Prérequis pour l'authentification sans clé :**

### Pour le développement local :
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Pour les environnements Azure :
- Activez **l'identité managée attribuée par le système** sur votre ressource Azure
- Attribuez les **rôles RBAC** appropriés à l'identité managée :
  - `Cognitive Services OpenAI User` pour l'accès à Azure OpenAI
  - `AI Developer` pour l'accès aux projets Azure AI

### Variables d'environnement (Optionnel) :
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Pas besoin de clés API ni de chaînes de connexion !** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Importer les bibliothèques nécessaires

Importer les modules Python requis :
- **os, time** : Bibliothèques standard de Python pour les variables d'environnement et les délais
- **AIProjectClient** : Client principal pour les projets Azure AI
- **DefaultAzureCredential** : Authentification sans clé pour les services Azure
- **Classes liées à MCP** : Pour créer et gérer les outils MCP et traiter les approbations


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Configurer les paramètres du serveur MCP

Configurez la configuration du serveur MCP en utilisant des variables d'environnement avec des valeurs par défaut de secours :
- **MCP_SERVER_URL** : L'URL du serveur MCP (par défaut, l'API Microsoft Learn)
- **MCP_SERVER_LABEL** : Un label pour identifier le serveur MCP (par défaut, "mslearn")

Cette approche permet une configuration flexible dans différents environnements.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Créer un client de projet Azure AI (Authentification sans clé)

Initialisez le client de projet Azure AI en utilisant **l'authentification sans clé** :
- **endpoint** : L'URL du point de terminaison du projet Azure AI Foundry
- **credential** : Utilise `DefaultAzureCredential()` pour une authentification sécurisée et sans clé
- **Aucune clé API requise** : Découvre et utilise automatiquement les meilleures informations d'identification disponibles

**Flux d'authentification :**
1. Vérifie l'identité managée (dans les environnements Azure)
2. Reprend les informations d'identification Azure CLI (pour le développement local)
3. Utilise d'autres sources d'informations d'identification disponibles si nécessaire

Cette approche élimine le besoin de gérer des clés API ou des chaînes de connexion dans votre code.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Créer une définition d'outil MCP

Créez un outil MCP qui se connecte au serveur MCP de Microsoft Learn :
- **server_label** : Identifiant pour le serveur MCP
- **server_url** : Point de terminaison URL du serveur MCP
- **allowed_tools** : Liste facultative pour restreindre les outils pouvant être utilisés (une liste vide permet l'utilisation de tous les outils)

Cet outil permettra à l'agent d'accéder à la documentation et aux ressources de Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Créer un agent et exécuter une conversation (Workflow sans clé)

Cette section complète illustre le **workflow d'agent sans clé** :

1. **Créer un agent IA** : Configurez un agent avec le modèle GPT-4.1 nano et les outils MCP.
2. **Créer un fil de discussion** : Établissez un fil de conversation pour la communication.
3. **Envoyer un message** : Posez une question à l'agent sur les différences entre Azure OpenAI et OpenAI.
4. **Gérer les approbations d'outils** : Approuvez automatiquement les appels aux outils MCP lorsque nécessaire.
5. **Surveiller l'exécution** : Suivez les progrès de l'agent et gérez les actions requises.
6. **Afficher les résultats** : Montrez les détails de la conversation et de l'utilisation des outils.

**Caractéristiques sans clé :**
- ✅ **Pas de secrets codés en dur** - Toute l'authentification est gérée par l'identité Azure.
- ✅ **Sécurisé par défaut** - Utilise un contrôle d'accès basé sur les rôles.
- ✅ **Déploiement simplifié** - Pas de gestion des identifiants nécessaire.
- ✅ **Compatible avec l'audit** - Tous les accès sont suivis via l'identité Azure.

L'agent utilisera les outils MCP pour accéder aux ressources Microsoft Learn avec une sécurité totale et sans gestion de clé API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
